### Setting up catalog with pics

How to import src without it?

In [1]:
import os
os.getcwd()
os.chdir('..')

### Import libraries

In [2]:
import os
import src.dataset.MonkeyDataset as md
import src.dataset.RandomImage as rI
from torchvision.transforms import ToPILImage
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib.pyplot import imshow
%matplotlib inline
from matplotlib import image
from matplotlib import pyplot
import pathlib
import IPython.display as display

### Download the files from Kaggle (run one time) and quick check of catague

In [ ]:
#md.MonkeyDataset('data', transform=None, phase='training', download=True)

In [ ]:
path = '/Users/martaplonka/Documents/DeepLearning/MonkeysKaggle/data/training/training'

In [ ]:
path = pathlib.Path(path)
image_count = len(list(path.glob('*/*.jpg')))
image_count

In [ ]:
second = list(path.glob('n2/*'))

In [ ]:
for image_path in second[:3]:
    display.display(Image.open(str(image_path)))

# Creating dataset with Andrew PyTorch

In [ ]:
dataset_train = md.MonkeyDataset('data', transform=None, phase='training', download=False) 

Przykladowy tensor dla obrazka, caly wiersz to tuple z tensora i kategorii. Ale nie widzi tego jako targetu, trzeba to chyba bedzie przerabiac

In [ ]:
dataset_train[0]

Obrazki maja rozne wielkosci

In [ ]:
print(dataset_train[2][0].shape)
print(dataset_train[0][0].shape)

In [ ]:
pixels = pd.DataFrame(columns = ['id', 'width', 'height'])
for i in range(len(dataset_train)):
    temp = pd.DataFrame([[i, dataset_train[i][0].shape[1], dataset_train[i][0].shape[2]]], 
                       columns = ['id', 'width', 'height'])
    pixels = pixels.append(temp, ignore_index= True)    

In [ ]:
pixels_agg = pixels.groupby(['width', 'height']).count().reset_index()

In [ ]:
plt.scatter(pixels_agg['width'], pixels_agg['height'], c=pixels_agg['id'], alpha=0.5)
plt.show()

### Showing random pic from DataSet

In [ ]:
rI.get_random_monkey(dataset_train, random = True, cat_num = None, pic_num = None )

# THE EXPLORATION WILL CONTINUE...

# LEARNING
## Import PIC, show PIC, info about PIC and convert to array 

### 1. Using MATPLOT LIB

Import image directly from the drive

In [ ]:
from PIL import Image
from matplotlib import image
from matplotlib.pyplot import imshow

In [ ]:
image2 = Image.open('data/training/training/n0/n018.jpg')

In [3]:
image2.format, image2.mode, image2.size

NameError: name 'image2' is not defined

In [ ]:
imshow(image2)

Convert to array

In [ ]:
data = image.imread('data/training/training/n0/n018.jpg')

In [ ]:
data[1]

In [ ]:
data.dtype, data.shape

Show pic based on array

In [ ]:
pyplot.imshow(data)
pyplot.show()

### 2. Using PILLOW

In [ ]:
from PIL import Image
from numpy import asarray

image = Image.open('data/training/training/n0/n0022.jpg')

In [ ]:
imshow(image)

In [ ]:
data = asarray(image)
data[1]

In [ ]:
data.shape

In [ ]:
image3 = Image.fromarray(data)
image3.format, image3.mode, image3.size

In [ ]:
imshow(image3)

#  LEARNING
## How to prepare data for modelling - Basic PILLOW

Czasami potrzebujemy zeby wystandaryzowac rozmiar zdjec

In [ ]:
from PIL import Image
image = Image.open('data/training/training/n0/n0022.jpg')

thumbnail zmienia rozmair zdjecia ale zachowuje aspect ratio oryginalnego obrazka

In [ ]:
print(image.size)
image.thumbnail((100,100))
print(image.size)

In [ ]:
imshow(image)

resize() ignoruje aspect ratio

In [ ]:
image = Image.open('data/training/training/n0/n0022.jpg')
print(image.size)
img_resized = image.resize((200,200))
print(img_resized.size)

In [ ]:
imshow(img_resized)

### Data augmentation

In [ ]:
from PIL import Image
from matplotlib import pyplot
image = Image.open('data/training/training/n0/n0022.jpg')

Flipowanie

In [ ]:
hoz_flip = image.transpose(Image.FLIP_LEFT_RIGHT)
ver_flip = image.transpose(Image.FLIP_TOP_BOTTOM)

In [ ]:
pyplot.subplot(311)
pyplot.imshow(image)
pyplot.subplot(312)
pyplot.imshow(hoz_flip)
pyplot.subplot(313)
pyplot.imshow(ver_flip)
pyplot.show()

Rotacja

In [ ]:
pyplot.subplot(311)
pyplot.imshow(image)
# rotate 45 degrees
pyplot.subplot(312)
pyplot.imshow(image.rotate(45))
# rotate 90 degrees
pyplot.subplot(313)
pyplot.imshow(image.rotate(90))
pyplot.show()

Cropping

In [ ]:
cropped = image.crop((100, 100, 1000, 1000))
imshow(cropped)

# PYTORCH HOW TO PREPARE DATA

In [ ]:
from torch.utils.data import Dataset
import torchvision as tv
import torch.utils.data as data
from torchvision import transforms, datasets
from torchvision.datasets import ImageFolder

In [ ]:
BATCH_SIZE = 1000

In [ ]:
preprocessing = transforms.Compose([
                transforms.RandomResizedCrop(256, scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(0, 1)
            ])

In [ ]:
train_data = datasets.ImageFolder('/Users/martaplonka/Documents/DeepLearning/MonkeysKaggle/data/training/training', transform = preprocessing)
train_data_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)

In [ ]:
test_data = datasets.ImageFolder('/Users/martaplonka/Documents/DeepLearning/MonkeysKaggle/data/validation/validation', transform = preprocessing)
test_data_loader = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)

In [ ]:
x, _ = train_data[10] 
imshow(x.numpy()[0])

In [ ]:
rI.get_random_monkey(train_data, random = True, cat_num = None, pic_num = None)

# KERAS HOW TO PREPARE DATA

Uzyc klasy ImageDataGenerator. Jej konstruktor zawiera argumenty do processingu obrazow jak pixels scaling i data augmentation.
The default ‘batch_size‘ is 32, which means that 32 randomly selected images from across the classes in the dataset will be returned in each batch when training. Larger or smaller batches may be desired. You may also want to return batches in a deterministic order when evaluating a model, which you can do by setting ‘shuffle‘ to ‘False.’
Wiecej info w: https://keras.io/api/preprocessing/image/

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
CLASS_NAMES = ['alouatta_palliata','erythrocebus_patas','cacajao_calvus','macaca_fuscata','cebuella_pygmea',
    'cebus_capucinus','mico_argentatus','saimiri_sciureus','aotus_nigriceps','trachypithecus_johnii']

In [ ]:
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE) #o co cho

Przeskalowanie wartosci RGB z integer do float32 range[0,1]

In [ ]:
datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_it = datagen.flow_from_directory(directory='data/training/training/', 
                                       class_mode='binary', 
                                       batch_size=BATCH_SIZE,
                                       shuffle=True,
                                       target_size=(IMG_HEIGHT, IMG_WIDTH))
val_it = datagen.flow_from_directory(directory='data/validation/validation/', 
                                     class_mode='binary',
                                     batch_size=BATCH_SIZE,
                                     shuffle=True,
                                     target_size=(IMG_HEIGHT, IMG_WIDTH))

Eksploracja batcha - funkcja z TensorFlow tutorials

In [ ]:
def show_batch(image_batch, label_batch, keras=True):
    plt.figure(figsize=(10,10))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        i = label_batch[n]
        if keras == False:
            i = i.decode("utf-8").replace('n', '')
            j = int(i)
        else:
            j = int(i)
        plt.title(CLASS_NAMES[j].title())
        plt.axis('off')

In [ ]:
image_batch, label_batch = next(train_it)
show_batch(image_batch, label_batch)

Wady korzystania z kerasa:
    - It's slow. See the performance section below.
    - It lacks fine-grained control.
    - It is not well integrated with the rest of TensorFlow.

Na przyszlosc:
 - czym rozni sie binary od categorical

# TENSOR FLOW HOW TO PREPARE DATA

In [ ]:
list_ds_train = tf.data.Dataset.list_files('data/training/training/*/*')

In [ ]:
for f in list_ds_train.take(1): print(f.numpy())

In [ ]:
def get_label(file_path):
    # convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    # return parts[-2] == CLASS_NAMES - to wtedy jako label zwraca wektor boolean i ma true na swojej labelce
    return parts[-2]

Funkcja do stworzenia tensor'a, rescalingu i standaryzacji wartosci RGB

In [ ]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

In [ ]:
def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

map tworzy dataset par image + label, troche jak lambda -> mapuje podana funckje do kazdego elementu dataset-u podanego - w tym przypadku listy plikow

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
labeled_ds_train = list_ds_train.map(process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
for image, label in labeled_ds_train.take(1):
    print(image.numpy().shape)
    print(label.numpy())

Zeby dataset byl dobry brakuje jeszcze:
    - To be well shuffled.
    - To be batched.
    - Batches to be available as soon as possible.

In [ ]:
labeled_ds_train = labeled_ds_train.shuffle(buffer_size=1000)
labeled_ds_train = labeled_ds_train.repeat()
labeled_ds_train = labeled_ds_train.batch(BATCH_SIZE)
# `prefetch` lets the dataset fetch batches in the background while the model
# is training.
train_ds = labeled_ds_train.prefetch(buffer_size=AUTOTUNE)

In [ ]:
image_batch, label_batch = next(iter(train_ds))
show_batch(image_batch.numpy(), label_batch.numpy(), keras=False)

Once the iterators have been prepared, we can use them when fitting and evaluating a deep learning model.

